In [38]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
import pickle
from tqdm import tqdm


In [29]:
manga = 'https://myanimelist.net/topmanga.php'

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

response = requests.get(manga, headers=headers)
print(response.status_code)

200


In [30]:
mangasoup = BeautifulSoup(response.text, "lxml")

In [32]:
mangasoup.find_all('a', attrs={'class': 'next'})[0]['href']

'?limit=50'

In [33]:
  
def get_links_to_movies(soup):
    h3s = soup.find_all('h3', attrs={'class': "manga_h3"})
    movie_links = {el.find('a').text: el.find('a')['href'] for el in h3s}
    return movie_links    

In [34]:
def get_link_to_next_page(soup):
    links = soup.find_all('a', attrs={'class': 'next'})
    if len(links) == 0:
        return None
    else:
        link = links[0]['href']
        return link

In [39]:

def get_all_links(url, max_links=30000):
    headers = {
        'Accept-Language': 'en',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'X-FORWARDED-FOR': '2.21.184.0'
    }
    
    all_movies_links = {}
    
    # Initialize tqdm progress bar
    pbar = tqdm(total=max_links, desc="Collecting movie links")
    
    while True:
        response = requests.get(url, headers=headers)
        assert response.status_code == 200
        soup = BeautifulSoup(response.text, 'html.parser')
        movie_links = get_links_to_movies(soup)
        movie_links = {movie: urllib.parse.urljoin(url, movie_link) for (movie, movie_link) in movie_links.items()}
        
        new_links_count = len(movie_links)
        all_movies_links.update(movie_links)
        
        # Update progress bar
        pbar.update(min(new_links_count, max_links - pbar.n))
        
        if len(all_movies_links) >= max_links:
            pbar.close()
            print(f'Reached {max_links} links. Stopping.')
            return dict(list(all_movies_links.items())[:max_links])
        
        link = get_link_to_next_page(soup)
        if not link:
            pbar.close()
            return all_movies_links
        url = urllib.parse.urljoin(url, link)
        time.sleep(1)


In [40]:

all_manga_links = get_all_links(manga)

Reached 30000 links. Stopping.


In [42]:
with open('all_manga_links', 'wb') as f:
    pickle.dump(all_manga_links, f)

In [45]:
all_manga_links['Ashita no Joe']

'https://myanimelist.net/manga/1303/Ashita_no_Joe'

get an example and scrap per page

In [54]:
Ashita = requests.get('https://myanimelist.net/manga/1303/Ashita_no_Joe', headers=headers)
print(Ashita.status_code)
ashita_soup = BeautifulSoup(Ashita.text)

200


In [60]:
# title in english
ashita_soup.find('span', attrs={'class': 'title-english'}).text

'Ashita no Joe: Fighting for Tomorrow'

In [56]:
# description
ashita_soup.find('span', attrs={'itemprop': 'description'}).text

"Joe Yabuki has learned how to toughen up and stop trusting others after a troubling youth spent bouncing between orphanages and fending off bullies. After finally escaping the cycle of violent temporary homes, Joe adopts a delinquent lifestyle in the streets of San'ya, a slum in Tokyo. He makes his way through the world with his fists, picking fights with anyone he pleases.\n\r\nDanpei Tange, a washed-up and alcoholic former boxing coach, notices Joe's animalistic combat ability and attempts to mentor the wild teenager. Despite developing an explosive relationship, the two form a close bond and rely on each other to look after the other homeless children in San'ya.\n\r\nUnable to abandon his destructive habits, Joe is arrested for petty crimes. In jail, he meets and fights with lifelong friends and rivals, including the boxing prodigy Tooru Rikiishi. Continuously challenging Rikiishi to fights, Joe is knocked out time after time due to his savage brawling style failing in the face of 

In [79]:
# take 1 review
ashita_soup.find_all('div', attrs={'class': 'text'})[3].text

"\n        Simply put, Ashita no Joe is nothing short of a timeless masterpiece of a manga. This really is likely to be one of the greatest manga creations that has ever been created as well as one of the greatest that will ever be created. In every single medium, whether it be in literature, film or anything else for that matter, there are a select few creations that manage to transcend the media in and off itself and become something truly epic, truly beautiful and truly classic. Ashita no Joe manages to pull of this staggering feat. It manages to go far beyond being just another\n                  ...\ninspirational story about one scrappy young lad's rise to the top of his particular sport. Ashita no Joe is a grand story about the entire life of it's main character, poignantly detailing all of the excruciating pain and suffering, expressing every thought and every action with moving and genuinely emotional realism. It is for this reason that anyone with even a modicum of admiration

In [80]:
# take 2 review
ashita_soup.find_all('div', attrs={'class': 'text'})[4].text

"\n        While I have this rated as a 10, I truly think it's a 9.5/10.\n\r\nThe story of Ashita no Joe is absolutely phenomenal, there's not a lot more that can be said.\n\r\nJoe as a man and his fights encompass almost everything that I find interesting about boxing and life in general. His fight with Rikishi haunting him forever was amazing, but I truly think that his fight against Mendoza was where Joe culminated and peaked, with a close second being his fight against Carlos. Joe is everything that tomorrow is.\n\r\nThe ONLY thing that I have a problem with, is that I think the start is a\n                  ...\nlittle slow, I didn't care too much for volume 2 and the first half of volume 3 as the juvie arc was a little boring. and the second complaint is that I feel as if youko being interested in yabuki is a forced dynamic for the magazine that doesn't come naturally. But genuinely, everything else is perfect. I love Ashita no Joe.\n"

In [81]:
# take 3 review
ashita_soup.find_all('div', attrs={'class': 'text'})[5].text

'\n        I considered not writing a review for Ashita no Joe, as I didn\'t finish it.  I didn\'t finish JoJo\'s Bizzare Adventure and don\'t feel I can meaningfully review it at this time.  But for Ashita no Joe, if I can assist one person in making a decision on this manga, maybe that\'ll be worth it. \n\r\nI came into this manga because many people bring it up when discussing Hajime No Ippo, but after trying to read it, I\'m just so disappointed in the series, that I wonder why the comparisons are made. \n\r\nThere\'s two big problems I have with Ashita no Joe that\n                  ...\nhave made it impossible to go on.  \n\r\nThe first is the art style, it feels like it\'s a comic strip style that was popular in America in the 1960s and 1970s, I often thought of the comic script Nancy for some reason.  When I think of Manga I usually think of a refined art style that feels impressive, and Ashita no Joe is absolutely not in that style.   It\'s a cheap exaggerated art style that w

In [193]:

def get_manga_text(manga_soup):
    # Get title
    # title = manga_soup.find('span', attrs={'class': 'title-english'})
    title = manga_soup.find('span', attrs={'itemprop': 'name'})
    title = title.get_text(strip=True) if title else np.nan

    # Get description
    description = manga_soup.find('span', attrs={'itemprop': 'description'})
    description = description.get_text(strip=True) if description else np.nan

    # Get the 4th review (index [3]) if it exists
    reviews = manga_soup.find_all('div', attrs={'class': 'text'})
    review_1 = reviews[3].get_text(strip=True) if len(reviews) > 3 else np.nan

    return [title, description, review_1]


In [194]:
def get_all_manga_text(all_manga_links, verbose=False):
    columns = ['title', 'description', 'review_1']
    data = []
    done = 0
    start = time.time()

    for movie, url in all_manga_links.items():
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                movies_stat = get_manga_text(soup)
            else:
                movies_stat = [np.nan] * 3

            data.append(movies_stat)

            if verbose:
                done += 1
                elapsed_time = time.time() - start
                print(f'Done with {done} out of {len(all_manga_links)}. Time spent: {elapsed_time:.1f} seconds')

            # Pause between requests to avoid hitting rate limits
            time.sleep(1 if response.status_code == 200 else 2)

        except requests.RequestException as e:
            print(f"Request failed for URL: {url}, Error: {e}")
            data.append([np.nan] * 3)
            time.sleep(2)  # Shorter sleep for failed requests

    # Create DataFrame once the data is gathered
    return pd.DataFrame(data=data, columns=columns)


In [195]:
temp = {}
for movie in list(all_manga_links.keys())[:3]:
    temp[movie] = all_manga_links[movie]

data = get_all_manga_text(temp, verbose=1)

Done with 1 out of 3. Time spent: 1.6 seconds
Done with 2 out of 3. Time spent: 4.1 seconds
Done with 3 out of 3. Time spent: 7.5 seconds


In [199]:
data.review_1[0]

"Story - 9.38The first three volumes may discourage some of the readers because it's starts off kind of slow and the initial artwork is not quite up to today's standards. However, after the reader begins the Golden Age arc it is nearly impossible to not be pulled in and become addicted to the series. It is chock-full of action, drama, character development, twists and turns, and just a dash of romance. It defies generalization and is just damn good storytelling.After the Gold Age arc wraps up the story come back full circle to where it left us at the end of volume 3. But by...then the rhythm of the manga has changed completely. The reader has established a deep connection with Guts, Casca, and Griffith and that only grows more as the story continues. You'll be confronted with conflicting emotions and think things like &quot;I really want to see Guts put Griffith in the ground&quot; and at the same time &quot;Griffith is just so damn cool&quot;. These conflicting thoughts and emotions j